In [14]:
# %%
# =============================================================================
# Célula 1: Imports e Configurações Globais
# =============================================================================
import pandas as pd
import numpy as np
import time
import os 
import matplotlib.pyplot as plt
import seaborn as sns # Para plots melhores (opcional)
import json # Para configs, embora não usado diretamente aqui
import traceback

# --- Imports da Biblioteca ---
try:
    from activetextclassification.data_preparation import load_and_prepare_data
    from activetextclassification.models import get_model, BaseTextClassifier, BaseFeatureClassifier
    from activetextclassification.embeddings import get_embedder, BaseEmbedder # Para embedder global
    from activetextclassification.optimization.genetic_l0_optimizer import GeneticL0Optimizer
    from activetextclassification.utils import preprocess_label # Para pré-processamento do df_full
    print("Módulos da biblioteca activetextclassification importados.")
except ModuleNotFoundError as e:
    print(f"ERRO CRÍTICO: Falha ao importar módulos: {e}. Verifique a instalação e o PYTHONPATH.")
    raise e
except Exception as e:
    print(f"ERRO inesperado durante importação: {e}")
    raise e


# --- Autoreload (Opcional, útil durante desenvolvimento) ---
try:
    %load_ext autoreload
    %autoreload 2
    print("Autoreload ativado.")
except Exception as e:
    print(f"Não foi possível ativar autoreload: {e}")


# --- Parâmetros do Experimento de Otimização ---
# <<< MUDE AQUI CONFORME NECESSÁRIO >>>
DATA_FILE = r'../data/dataset.csv'
TEXT_COLUMN = 'nm_item'
LABEL_COLUMN = 'nm_product' # Label "perfeito" original
MIN_SAMPLES_PREPROC = 5     # Mínimo de amostras para uma classe não ser agrupada
RARE_LABEL_PREPROC = "_RARE_" # Rótulo para classes raras

# Configuração do Classificador (o mesmo usado na função de fitness do AG)
CLASSIFIER_CONFIG_AG = {
    'type': 'PVBin', # ProductVectorizer Binário
    'params': {'method':'binary', 'query':'binary', 'norm':None, 'query_norm':None, 'ngram_range': [1,2]}
}
# Se o classificador for baseado em features (ex: 'GNB', 'LSVC'), defina o GLOBAL_EMBEDDER_CONFIG_AG
GLOBAL_EMBEDDER_CONFIG_AG = None # Exemplo:
# GLOBAL_EMBEDDER_CONFIG_AG = {
#     'type': 'PVProb', # Usar ProductVectorizerEmbedder para gerar features
#     'params': {
#         'vectorizer_params': {'method':'tfidf', 'query':'tfidf', 'norm':'l2', 'query_norm':'l2', 'ngram_range': (1,1)},
#         'cache_dir': ".pv_cache_ag_embedder"
#     }
# }

# Parâmetros do Algoritmo Genético
L0_SIZE_TO_OPTIMIZE = 100 # Tamanho do L0 que queremos encontrar (ex: 100, 500, 1000)
POPULATION_SIZE_AG = 20    # Número de indivíduos na população (ex: 20-50)
N_GENERATIONS_AG = 100      # Número de gerações (ex: 10-100)
CROSSOVER_RATE_AG = 0.8
MUTATION_RATE_AG = 0.2    # Probabilidade de um indivíduo sofrer mutação
MUTATION_STRENGTH_AG = 4  # Quantos "genes" (índices) trocar na mutação
ELITISM_RATE_AG = 0.2     # 10% da elite passa para a próxima geração
TOURNAMENT_SIZE_AG = 3    # Tamanho do torneio para seleção

# Arquivos de Saída (searão prefixados com MAX_ ou MIN_)
OPTIMIZATION_HISTORY_BASE_FILE = f"ag_l0_opt_history_size{L0_SIZE_TO_OPTIMIZE}minmax"
BEST_L0_BASE_FILE = f"ag_l0_opt_best_l0_size{L0_SIZE_TO_OPTIMIZE}minmax"
DETAILED_FITNESS_LOG_BASE_FILE = f"ag_detailed_fitness_size{L0_SIZE_TO_OPTIMIZE}minmax"
# <<< FIM MUDE AQUI >>>

print(f"Experimento de Otimização de L0 com Algoritmo Genético")
print(f"Dataset: {DATA_FILE}, L0 Size: {L0_SIZE_TO_OPTIMIZE}")
print(f"Classificador para Fitness: {CLASSIFIER_CONFIG_AG['type']}")
if GLOBAL_EMBEDDER_CONFIG_AG: print(f"Embedder Global para Fitness: {GLOBAL_EMBEDDER_CONFIG_AG['type']}")
print(f"AG Params: Pop={POPULATION_SIZE_AG}, Gens={N_GENERATIONS_AG}")

Módulos da biblioteca activetextclassification importados.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Autoreload ativado.
Experimento de Otimização de L0 com Algoritmo Genético
Dataset: ../data/dataset.csv, L0 Size: 100
Classificador para Fitness: PVBin
AG Params: Pop=20, Gens=100


In [15]:
# %%
# =============================================================================
# Célula 2: Carregar Dados e Preparar Embedder Global (se necessário)
# =============================================================================
print("\n--- Carregando e Preparando Dataset Completo para AG ---")
df_full_ag = None
all_possible_labels_ag = []
embedder_instance_ag = None # Para guardar o embedder global se usado

try:
    # Usar load_and_prepare para obter labels únicos e fazer pré-proc inicial
    _, _, _, _, all_possible_labels_ag = load_and_prepare_data(
        file_path=DATA_FILE, text_column=TEXT_COLUMN, label_column=LABEL_COLUMN,
        min_samples_per_class=MIN_SAMPLES_PREPROC, rare_group_label=RARE_LABEL_PREPROC,
        population_size=0.99 # Apenas para carregar e processar quase tudo
    )
    if not all_possible_labels_ag: raise ValueError("Nenhum label encontrado após preparação inicial.")
    print(f"Labels únicos finais ({len(all_possible_labels_ag)}): {all_possible_labels_ag[:10]}...")

    # Recarregar o DF completo e aplicar o mesmo pré-processamento de labels
    print(f"Recarregando DataFrame completo de: {DATA_FILE}")
    file_ext_ag = os.path.splitext(DATA_FILE)[1].lower()
    if file_ext_ag == '.csv': df_full_ag = pd.read_csv(DATA_FILE)
    elif file_ext_ag in ['.xlsx', '.xls']: df_full_ag = pd.read_excel(DATA_FILE)
    else: raise ValueError(f"Extensão de arquivo não suportada: {file_ext_ag}")

    df_full_ag.dropna(subset=[TEXT_COLUMN, LABEL_COLUMN], inplace=True)
    # Aplicar preprocess_label e agrupamento de raros consistentemente
    df_full_ag[LABEL_COLUMN] = df_full_ag[LABEL_COLUMN].apply(preprocess_label).astype(str)
    if MIN_SAMPLES_PREPROC > 1:
        label_counts_ag = df_full_ag[LABEL_COLUMN].value_counts()
        rare_labels_ag = label_counts_ag[label_counts_ag < MIN_SAMPLES_PREPROC].index.tolist()
        if rare_labels_ag:
            print(f"Aplicando agrupamento de {len(rare_labels_ag)} classes raras em '{RARE_LABEL_PREPROC}' no DF completo...")
            df_full_ag[LABEL_COLUMN] = df_full_ag[LABEL_COLUMN].replace(rare_labels_ag, RARE_LABEL_PREPROC)

    # Filtrar o DF completo para conter apenas os labels finais válidos
    df_full_ag = df_full_ag[df_full_ag[LABEL_COLUMN].isin(all_possible_labels_ag)].copy()
    df_full_ag.reset_index(drop=True, inplace=True) # Crucial para AG usar índices de 0 a N-1

    print(f"Dataset AG preparado. Total de amostras: {len(df_full_ag)}")
    if df_full_ag.empty: raise ValueError("Dataset AG vazio após preparação.")
    if L0_SIZE_TO_OPTIMIZE > len(df_full_ag):
         raise ValueError(f"L0_SIZE_TO_OPTIMIZE ({L0_SIZE_TO_OPTIMIZE}) é maior que o dataset filtrado ({len(df_full_ag)}).")


    # Preparar embedder global SE o classificador AG for BaseFeatureClassifier
    clf_type_ag = CLASSIFIER_CONFIG_AG.get('type')
    if clf_type_ag in ['GNB', 'LSVC', 'LR', 'SGD'] and GLOBAL_EMBEDDER_CONFIG_AG:
        print("\nPreparando embedder global para AG (usado pela função de fitness)...")
        embedder_instance_ag = get_embedder(GLOBAL_EMBEDDER_CONFIG_AG)
        # O fit do embedder precisa de todos os textos e labels para consistência
        # (especialmente para PVProbEmbedder)
        embedder_instance_ag.fit(df_full_ag[TEXT_COLUMN].tolist(), df_full_ag[LABEL_COLUMN].tolist())
        print("Embedder global para AG ajustado.")
    elif clf_type_ag in ['GNB', 'LSVC', 'LR', 'SGD'] and not GLOBAL_EMBEDDER_CONFIG_AG:
        raise ValueError(f"Classificador AG '{clf_type_ag}' requer GLOBAL_EMBEDDER_CONFIG_AG, mas não foi fornecido.")


except Exception as e:
    print(f"ERRO CRÍTICO ao carregar/preparar dados para AG: {e}")
    traceback.print_exc()
    # Parar o notebook se os dados não puderem ser carregados
    raise SystemExit("Falha na preparação dos dados para o AG.")


--- Carregando e Preparando Dataset Completo para AG ---
--- Iniciando Preparação de Dados ---
Carregando dados de: ../data/dataset.csv
Shape inicial: (250365, 2)
Shape após remover NaNs: (250365, 2) (0 linhas removidas)
Pré-processando rótulos...
Verificando classes com menos de 5 amostras...
Agrupando 174 classes raras em '_RARE_': ['soquete', 'jabuticaba', 'jogo de carta', 'farinha sem gluten', 'escada', 'maria mole', 'impressora', 'analgesico veterinario', 'estojo', 'quadro']...
Distribuição de classes após agrupamento:
nm_product
biscoito                                 14292
iogurte                                   7520
chocolate                                 6732
padaria e confeitaria industrializado     6046
aperitivo                                 5186
Name: count, dtype: int64
Mapeamento de rótulos finalizado: 622 classes (incluindo '_RARE_' se criado).
Dividindo em População (P = 99%) e Pool (U)...
Tamanho População P: 247861
Tamanho Pool U inicial: 2504
--- Preparação 

In [10]:
def run_and_save_optimization(df_full, optimization_goal, fitness_metric_to_optimize, seed_offset=0):
    """
    Executa o AG para um dado objetivo e salva os resultados.
    """
    print(f"\n{'='*15} Iniciando Otimização de L0 (Tamanho: {L0_SIZE_TO_OPTIMIZE}) com AG - Objetivo: {optimization_goal.upper()} - Métrica: {fitness_metric_to_optimize} {'='*15}")

    ag_optimizer = GeneticL0Optimizer(
        df_full=df_full,
        text_column=TEXT_COLUMN,
        label_column=LABEL_COLUMN,
        classifier_config=CLASSIFIER_CONFIG_AG,
        initial_l0_size=L0_SIZE_TO_OPTIMIZE,
        all_possible_labels=all_possible_labels_ag,
        population_size=POPULATION_SIZE_AG,
        n_generations=N_GENERATIONS_AG,
        crossover_rate=CROSSOVER_RATE_AG,
        mutation_rate=MUTATION_RATE_AG,
        mutation_strength=MUTATION_STRENGTH_AG,
        elitism_rate=ELITISM_RATE_AG,
        fitness_metric=fitness_metric_to_optimize, # Métrica alvo
        optimization_goal=optimization_goal,       # 'maximize' ou 'minimize'
        tournament_size=TOURNAMENT_SIZE_AG,
        random_seed=42 + seed_offset, # Seed diferente para cada otimização
        embedder=embedder_instance_ag,
        log_detailed_fitness=True,
        detailed_log_file=f"{DETAILED_FITNESS_LOG_BASE_FILE}_{fitness_metric_to_optimize.split('_')[0].upper()}_{optimization_goal.upper()}.csv"
    )

    best_l0_indices, best_actual_perf, history_df = ag_optimizer.run_optimization()

    # --- Salvar Resultados da Otimização ---
    goal_suffix = f"_{fitness_metric_to_optimize.split('_')[0].upper()}_{optimization_goal.upper()}"

    # 1. Salvar Histórico da Otimização (fitness por geração)
    if history_df is not None and not history_df.empty:
        file_hist = f"{OPTIMIZATION_HISTORY_BASE_FILE}{goal_suffix}.xlsx"
        print(f"Salvando histórico da otimização ({goal_suffix}) em: {file_hist}")
        try: history_df.to_excel(file_hist, index=False)
        except Exception as e: print(f"Erro ao salvar histórico AG ({goal_suffix}): {e}")
    else:
        print(f"Nenhum histórico de otimização para salvar ({goal_suffix}).")

    # 2. Salvar o Melhor/Pior L0 Encontrado
    if best_l0_indices is not None and len(best_l0_indices) > 0:
        file_best_l0 = f"{BEST_L0_BASE_FILE}{goal_suffix}.csv"
        print(f"Salvando L0 (Objetivo: {optimization_goal}, Perf: {best_actual_perf:.4f}) em: {file_best_l0}")
        best_l0_df = df_full.iloc[best_l0_indices][[TEXT_COLUMN, LABEL_COLUMN]].copy()
        best_l0_df['metric_value'] = best_actual_perf # Performance real
        best_l0_df['metric_type'] = fitness_metric_to_optimize
        best_l0_df['optimization_goal'] = optimization_goal
        try: best_l0_df.to_csv(file_best_l0, index=False, encoding='utf-8-sig')
        except Exception as e: print(f"Erro ao salvar melhor/pior L0 ({goal_suffix}): {e}")
    else:
        print(f"Nenhum L0 encontrado para o objetivo '{optimization_goal}' ({goal_suffix}).")

    return best_l0_indices, best_actual_perf, history_df



In [ ]:
# %%
# =============================================================================
# Célula 4: Executar Otimizações (Maximizar e Minimizar para cada Métrica)
# =============================================================================

# Resultados das otimizações serão armazenados aqui
optimization_results = {}

# --- Otimizar para ACURÁCIA ---
# Maximizar Acurácia
res_acc_max = run_and_save_optimization(df_full_ag, 'maximize', 'accuracy_on_full', seed_offset=0)
optimization_results['ACC_MAX'] = {'indices': res_acc_max[0], 'performance': res_acc_max[1], 'history': res_acc_max[2]}
print(optimization_results)
# Minimizar Acurácia
res_acc_min = run_and_save_optimization(df_full_ag, 'minimize', 'accuracy_on_full', seed_offset=100) # Seed diferente
optimization_results['ACC_MIN'] = {'indices': res_acc_min[0], 'performance': res_acc_min[1], 'history': res_acc_min[2]}


# --- Otimizar para F1-SCORE ---
# Maximizar F1-Score
res_f1_max = run_and_save_optimization(df_full_ag, 'maximize', 'f1_macro_on_full', seed_offset=200)
optimization_results['F1_MAX'] = {'indices': res_f1_max[0], 'performance': res_f1_max[1], 'history': res_f1_max[2]}
print(optimization_results)
# Minimizar F1-Score
res_f1_min = run_and_save_optimization(df_full_ag, 'minimize', 'f1_macro_on_full', seed_offset=300)
optimization_results['F1_MIN'] = {'indices': res_f1_min[0], 'performance': res_f1_min[1], 'history': res_f1_min[2]}

print("\n--- Todas as Otimizações Concluídas ---")


=============== Iniciando Otimização de L0 (Tamanho: 100) com AG - Objetivo: MAXIMIZE - Métrica: accuracy_on_full ===============
GeneticL0Optimizer inicializado.
 - População: 20, Gerações: 100, L0 Size: 100
 - Objetivo: maximize, Métrica: accuracy_on_full
 - Log Detalhado ATIVADO em: d:\Nuvem\ghdaru\OneDrive\030_DOUTORADO\120_TESE\130_TESEGIT\Tese-Vers-o-Draft\code\codigos\falco_library\examples\ag_detailed_fitness_size100minmax_ACCURACY_MAXIMIZE.csv

--- Iniciando Otimização Genética (Tam: 100, Obj: maximize, Métrica: accuracy_on_full) ---
Inicializando população...


Criando População Inicial:   0%|          | 0/20 [00:00<?, ?it/s]

Gerações AG:   0%|          | 0/100 [00:00<?, ?it/s]


  Geração 1/100


Fitness Calc Gen 1:   0%|          | 0/20 [00:00<?, ?it/s]

...

Model Factory: Criando tipo 'PVBin' com params: {'method': 'binary', 'query': 'binary', 'norm': None, 'query_norm': None, 'ngram_range': [1, 2]}
Model Factory (PVBin): Convertido ngram_range [1, 2] para tupla (1, 2)
Fitting ProductVectorizerClassifier com 100 amostras...
Fit concluído.
Model Factory: Criando tipo 'PVBin' com params: {'method': 'binary', 'query': 'binary', 'norm': None, 'query_norm': None, 'ngram_range': [1, 2]}
Model Factory (PVBin): Convertido ngram_range [1, 2] para tupla (1, 2)
Fitting ProductVectorizerClassifier com 100 amostras...
Fit concluído.
Model Factory: Criando tipo 'PVBin' com params: {'method': 'binary', 'query': 'binary', 'norm': None, 'query_norm': None, 'ngram_range': [1, 2]}
Model Factory (PVBin): Convertido ngram_range [1, 2] para tupla (1, 2)
Fitting ProductVectorizerClassifier com 100 amostras...
Fit concluído.
Model Factory: Criando tipo 'PVBin' com params: {'method': 'binary', 'query': 'binary', 'norm': None, 'query_norm': None, 'ngram_range': [1

Fitness Calc Gen 40:   0%|          | 0/20 [00:00<?, ?it/s]

Model Factory: Criando tipo 'PVBin' com params: {'method': 'binary', 'query': 'binary', 'norm': None, 'query_norm': None, 'ngram_range': [1, 2]}
Model Factory (PVBin): Convertido ngram_range [1, 2] para tupla (1, 2)
Fitting ProductVectorizerClassifier com 100 amostras...
Fit concluído.
Model Factory: Criando tipo 'PVBin' com params: {'method': 'binary', 'query': 'binary', 'norm': None, 'query_norm': None, 'ngram_range': [1, 2]}
Model Factory (PVBin): Convertido ngram_range [1, 2] para tupla (1, 2)
Fitting ProductVectorizerClassifier com 100 amostras...
Fit concluído.
Model Factory: Criando tipo 'PVBin' com params: {'method': 'binary', 'query': 'binary', 'norm': None, 'query_norm': None, 'ngram_range': [1, 2]}
Model Factory (PVBin): Convertido ngram_range [1, 2] para tupla (1, 2)
Fitting ProductVectorizerClassifier com 100 amostras...
Fit concluído.
Model Factory: Criando tipo 'PVBin' com params: {'method': 'binary', 'query': 'binary', 'norm': None, 'query_norm': None, 'ngram_range': [1

Fitness Calc Gen 41:   0%|          | 0/20 [00:00<?, ?it/s]

Model Factory: Criando tipo 'PVBin' com params: {'method': 'binary', 'query': 'binary', 'norm': None, 'query_norm': None, 'ngram_range': [1, 2]}
Model Factory (PVBin): Convertido ngram_range [1, 2] para tupla (1, 2)
Fitting ProductVectorizerClassifier com 100 amostras...
Fit concluído.


In [ ]:
# %%
# =============================================================================
# Célula 5: Plotar Evolução do Fitness de Todas as Otimizações
# =============================================================================
print("\n--- Plotando Evolução do Fitness (AG) para Todas as Otimizações ---")

# Configurar o backend de plotagem (se não estiver configurado na Célula 1)
# %matplotlib inline # Se estiver rodando em Jupyter Notebook / Lab
# ou
# %matplotlib notebook # Se quiser gráficos interativos (pode precisar instalar widgets)


num_optimizations = len(optimization_results)
if num_optimizations > 0:
    # Verificar se há pelo menos um histórico válido para plotar
    has_valid_history = any(result.get('history') is not None and not result['history'].empty for result in optimization_results.values())

    if has_valid_history:
        # Determinar o número de linhas e colunas para os subplots (2x2 para Acc/F1 vs Max/Min)
        fig, axes = plt.subplots(2, 2, figsize=(18, 10), sharey=False) # 2x2 grid
        axes_flat = axes.flatten() # Para facilitar a iteração
        plot_idx = 0 # Índice para percorrer os eixos

        # Iterar sobre os resultados das otimizações
        for key, result in optimization_results.items():
            history_df = result.get('history') # Usar .get() para segurança
            goal = "Maximizar" if "MAX" in key else "Minimizar"
            metric_base_name = "Acurácia" if "ACC" in key else "F1-Macro"
            title = f"AG {goal} {metric_base_name}\nL0 Tam: {L0_SIZE_TO_OPTIMIZE}"


            # --- VERIFICAR SE HÁ DADOS VÁLIDOS PARA ESTE PLOT ESPECÍFICO ---
            plot_this_one = False
            if history_df is not None and not history_df.empty:
                 # Verificar se as colunas de plotagem existem e contêm pelo menos um valor não-NaN/inf
                 cols_to_check = []
                 if "ACC" in key: cols_to_check = ['max_acc', 'avg_acc', 'min_acc']
                 elif "F1" in key: cols_to_check = ['max_f1', 'avg_f1', 'min_f1']

                 if all(col in history_df.columns for col in cols_to_check):
                      # Verifica se alguma linha tem dados válidos em qualquer uma das colunas relevantes
                      if not history_df[cols_to_check].dropna(how='all').empty:
                           plot_this_one = True
                      else:
                           print(f"AVISO Plot: Dados de histórico para '{key}' existem, mas todas as colunas de plotagem estão vazias/NaN/Inf.")
                 else:
                      print(f"AVISO Plot: Colunas de plotagem {cols_to_check} ausentes no histórico para '{key}'.")


            # --- REALIZAR A PLOTAGEM SE HOUVER DADOS ---
            if plot_this_one and plot_idx < len(axes_flat):
                ax = axes_flat[plot_idx]

                if "ACC" in key:
                    ax.plot(history_df['generation'], history_df['max_acc'], label='Max Acc (Real)')
                    ax.plot(history_df['generation'], history_df['avg_acc'], label='Avg Acc (Real)')
                    ax.plot(history_df['generation'], history_df['min_acc'], label='Min Acc (Real)', linestyle='--')
                    ax.set_ylabel(f'Acurácia (Real)')
                elif "F1" in key:
                    ax.plot(history_df['generation'], history_df['max_f1'], label='Max F1 (Real)')
                    ax.plot(history_df['generation'], history_df['avg_f1'], label='Avg F1 (Real)')
                    ax.plot(history_df['generation'], history_df['min_f1'], label='Min F1 (Real)', linestyle='--')
                    ax.set_ylabel(f'F1-Macro (Real)')

                ax.set_xlabel('Geração')
                ax.set_title(title)
                ax.legend()
                ax.grid(True)
                plot_idx += 1
            # --- Adicionar placeholder nos eixos se não houver dados ---
            elif plot_idx < len(axes_flat):
                 ax = axes_flat[plot_idx]
                 ax.set_title(f"{title}\n(Sem Dados Válidos para Plotar)")
                 ax.text(0.5, 0.5, "Verificar logs detalhados/erros.", ha='center', va='center', wrap=True)
                 ax.set_xlabel('Geração') # Manter labels do eixo
                 ax.set_ylabel(f'{metric_base_name} (Real)')
                 ax.grid(True) # Manter grid
                 plot_idx += 1


        # Remover eixos não utilizados (se menos de 4 plots foram feitos)
        for i in range(plot_idx, len(axes_flat)):
            fig.delaxes(axes_flat[i])

        # Ajustar layout e mostrar figura
        fig.suptitle(f'Evolução do Fitness - L0 Tamanho {L0_SIZE_TO_OPTIMIZE} - Classificador: {CLASSIFIER_CONFIG_AG["type"]}', fontsize=16, y=1.02)
        plt.tight_layout(rect=[0, 0, 1, 0.98]) # Ajustar para o super título
        plt.show() # Forçar a exibição

    else:
        print("Nenhum resultado de otimização válido (status 'COMPLETED' com history_data) encontrado para plotar.")

else:
    print("\nDataFrame 'optimization_results' vazio. Nenhuma otimização foi executada.")

In [ ]:
optimization_results

{'ACC_MAX': {'indices': array([185856, 182403, 229129, 217098, 198927, 223761, 244377, 149274,
         125979, 185754,  38299, 204317,   6687, 237984,  48420, 240549,
          73896,  38955,  96043, 101803, 179117, 209454, 229677,  72370,
         103474,   8119,  71735, 236473, 189369, 104379, 208324,  92234,
         236490, 204749,  39631,  95313, 139217, 138451, 130006, 185818,
         106743,  79197,  67933,  36588,  65015, 226426, 189692,   8317,
         237823,  84343], dtype=int64),
  'performance': 0.3176562219160026,
  'history':     generation   max_acc   avg_acc   min_acc    max_f1    avg_f1    min_f1  \
  0            1  0.216795  0.174488  0.107882  0.021796  0.017756  0.013497   
  1            2  0.224544  0.197702  0.166553  0.021796  0.018034  0.015705   
  2            3  0.226250  0.212865  0.194891  0.021185  0.018713  0.016107   
  3            4  0.231889  0.210617  0.176510  0.021092  0.018393  0.014689   
  4            5  0.231889  0.218476  0.203970  0.02